In [ ]:
!python3.11 -m pip install langchain faiss-cpu langchain-openai pandas

In [ ]:
import pandas as pd

reports = pd.read_csv('./reports.tsv', sep='\t')[['quarter', 'report']]
reports

In [ ]:
from langchain.docstore.document import Document

documents = [ Document(page_content=report[2].strip(), metadata={'quarter': report[1]}) for report in reports.itertuples() ]
documents

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='. ', chunk_size=500, chunk_overlap=50, is_separator_regex=False)

split_documents = text_splitter.split_documents(documents)
split_documents

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-l6-v2')
db = FAISS.from_documents(split_documents, embeddings)

In [ ]:
question = 'What is the return on investment in Q4 2023'
docs = db.similarity_search_with_score(question)

docs

In [ ]:
prompt = f'Context: { "; ".join([ doc[0].page_content for doc in docs ]) }, Question: {question}. Answer the phrased question using exclusively the provided information in the context.'
prompt

In [ ]:
key = '...'
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=key, model_name='gpt-3.5-turbo')

In [ ]:
llm.invoke(prompt)